# Full Search

## Load Bible Text

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hanziconv import HanziConv
from data import read_bible

unv = read_bible('data/dnstrunv.tgz')
unv['text_s'] = unv.text.apply(HanziConv.toSimplified)

## Tokenize

In [3]:
import jieba
import jieba.posseg as pseg

jieba.load_userdict('bible_terms.txt')

unv['text_tk'] = unv.text_s.apply(pseg.lcut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w0/lx0qzxln5p72qnrg392g7nyc0000gn/T/jieba.cache
Loading model cost 2.007 seconds.
Prefix dict has been built successfully.


## Search

In [4]:
from typing import Iterable

def highlight_occurances(text: str, keywords: Iterable[str]) -> str:
    for i, kw in enumerate(keywords):
        text = text.replace(kw, highlight(kw, i))
    return text

def highlight(text: str, color_code: int) -> str:
    return f'\x1b[6;30;4{color_code + 1}m{text}\x1b[0m'

test_data = ['first', 'second', 'thrid', 'forth', 'fifth', 'sixth', 'seventh']
print(highlight_occurances(', '.join(test_data), test_data))

first, second, thrid, forth, fifth, sixth, seventh


In [8]:
from search import sentence_similarity

searches = ['欢乐 祈祷', '挂虑 祈祷', '虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

for search_term in searches:
    search_tk = pseg.lcut(search_term)
    print(f'Search for {search_term} ({search_tk}):')
    match_scores = {}
    for v, vers_tk in zip(unv.text_s, unv.text_tk):
        similarity = sentence_similarity(search_tk, vers_tk)
        match_kw = [kw for kw in similarity.keys() if similarity[kw] > 0]
        vers = highlight_occurances(v, match_kw)
        score = sum(similarity.values())
        match_scores[vers] = score
    for top_match in sorted(match_scores, key=match_scores.get, reverse=True)[:10]:
        print(f'Match: {match_scores[top_match]:7.4f} Verse: {top_match}')
    print()

Search for 欢乐 祈祷 ([pair('欢乐', 'a'), pair(' ', 'x'), pair('祈祷', 'v')]):
Match:  1.2500 Verse: 耶何耶大派官看守耶和华的殿，是在祭司利未人手下。这祭司利未人是大卫分派在耶和华殿中、照摩西律法上所写的，给耶和华献燔祭，又按大卫所定的例，欢乐歌唱；
Match:  1.2500 Verse: 我向耶和华─我的神祈祷、认罪，说：「主啊，大而可畏的神，向爱主、守主诫命的人守约施慈爱。
Match:  1.2500 Verse: 就祷告耶和华说：「耶和华啊，我在本国的时候岂不是这样说吗？我知道你是有恩典、有怜悯的神，不轻易发怒，有丰盛的慈爱，并且后悔不降所说的灾，所以我急速逃往他施去。
Match:  1.2500 Verse: 耶稣在一个地方祷告；祷告完了，有个门徒对他说：「求主教导我们祷告，像约翰教导他的门徒。」
Match:  1.2500 Verse: 白昼，耶和华必向我施慈爱；黑夜，我要歌颂祷告赐我生命的神。
Match:  1.2500 Verse: 神是应当称颂的！他并没有推却我的祷告，也没有叫他的慈爱离开我。
Match:  1.2500 Verse: 但我在悦纳的时候向你─耶和华祈祷。神啊，求你按你丰盛的慈爱，凭你拯救的诚实应允我！
Match:  1.2000 Verse: 耶和华神啊，求你起来，和你有能力的约柜同入安息之所。耶和华神啊，愿你的祭司披上救恩；愿你的圣民蒙福欢乐。
Match:  1.2000 Verse: 看见王站在殿门的柱旁，百夫长和吹号的人侍立在王左右，国民都欢乐吹号，又有歌唱的，用各样的乐器领人歌唱赞美；亚她利雅就撕裂衣服，喊叫说：「反了！反了！」
Match:  1.2000 Verse: 看见王照例站在柱旁，百夫长和吹号的人侍立在王左右，国中的众民欢乐吹号；亚她利雅就撕裂衣服，喊叫说：「反了！反了！」

Search for 挂虑 祈祷 ([pair('挂虑', 'v'), pair(' ', 'x'), pair('祈祷', 'v')]):
Match:  2.0000 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉神。
Match:  1.2000 Verse: 妇人和处女也有分别。没有出嫁的，是为主的事挂虑